In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")


True

## Setup Tools


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

## Create State

In [5]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

## Create Subagents


In [6]:
from langchain.agents import create_agent

# Travel agent
travel_agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt="""
    **여행사(트래블 에이전트)**입니다. 사용자가 원하는 **결혼식 목적지(웨딩 장소)**로 가는 항공편을 검색하세요.
    다음 조건에 따라 최적의 항공편 옵션을 찾아야 하며, 추가 질문(후속 질문)은 절대 하면 안 됩니다. 아래 기준을 종합해 가장 좋은 항공편 옵션을 찾으세요.
    - 가격: 가장 저렴한 항공권(이코노미 클래스)
    - 소요 시간: 가장 짧은 총 이동 시간(최단)
    - 날짜: 해당 목적지에서 결혼식을 진행하기에 가장 적합하다고 판단되는 계절/시기
    
    작업을 단순화하기 위해 다음을 준수하세요.
    - 항공권 1장
    - 편도(one-way)
    - 성인 1인 기준
    
    최적의 결과를 얻기 위해 여러 번 검색을 반복하며 후보를 좁혀도 됩니다.
    사용자에게는 추가 정보가 제공되지 않습니다. 오직 **출발지(origin)**와 **도착지(destination)**만 주어집니다. 주어진 정보만으로 비판적으로 판단하여 최선의 항공편을 찾아내는 것이 당신의 역할입니다.
    최적의 결과를 찾은 뒤, 사용자에게 **추천 항공편 후보(쇼트리스트)**를 정리해 알려주세요.
    """
)

In [7]:
# Venue agent
venue_agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt="""
    당신은 행사장(베뉴) 전문 컨설턴트입니다. 사용자가 원하는 지역과 수용 인원에 맞는 행사장(베뉴)을 검색하세요.
    추가 질문(후속 질문)은 절대 하면 안 됩니다. 아래 기준을 바탕으로 가장 좋은 행사장 옵션을 반드시 찾아야 합니다.
    - 가격: 가장 저렴한 곳(최저가)
    - 수용 인원: 사용자가 원하는 인원과 정확히 일치(정원 정확 일치)
    - 리뷰: 평점/후기가 가장 좋은 곳(최고 평가)
    최적의 결과를 얻기 위해 여러 번 검색을 반복하며 후보를 점진적으로 좁혀도 됩니다.
    """
)

In [8]:
# Playlist agent
playlist_agent = create_agent(
    model="gpt-5-nano",
    tools=[query_playlist_db],
    system_prompt="""
    당신은 플레이리스트(음악 큐레이션) 전문가입니다. 주어진 **장르(genre)**를 기반으로 SQL 데이터베이스를 조회하여 결혼식에 어울리는 완벽한 플레이리스트를 큐레이션하세요.
    플레이리스트를 구성한 뒤에는 다음을 수행해야 합니다.
    - 플레이리스트의 **총 재생 시간(전체 곡 길이 합계)**을 계산하세요.
    - 각 곡에는 가격이 연결되어 있으므로, 플레이리스트의 **총 비용(전체 곡 가격 합계)**도 계산하세요.
    
    데이터베이스를 조회하는 과정에서 오류가 발생하면 쿼리의 문법/조건/구조를 수정하여 오류를 해결하고 다시 시도하세요.
    빈손으로 돌아오면 안 됩니다.
    - 반드시 곡 리스트를 찾을 때까지 데이터베이스 조회를 계속 시도하세요.
    - 결과가 나올 때까지 쿼리를 반복적으로 개선하며 재조회하세요.
    최적의 결과를 얻기 위해 여러 번의 SQL 쿼리를 수행하면서 후보를 점진적으로 좁혀도 됩니다.
    """
)

## Main Coordinator


In [9]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )


In [10]:

from langchain.agents import create_agent

coordinator = create_agent(
    model="gpt-5-nano",
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    당신은 웨딩 코디네이터입니다. 항공권, 행사장(베뉴), 플레이리스트를 담당하는 전문가들에게 업무를 위임하세요.
    먼저 상태(state)를 업데이트하기 위해 필요한 모든 정보를 수집하세요. 
    추가 질문(후속 질문)은 절대 하면 안 됩니다.
    필요한 정보 수집이 완료된 후 각 전문가에게 업무를 위임할 수 있습니다.
    각 전문가로부터 답변을 받은 뒤, 그 결과를 종합하여 나를 위한 완벽한 결혼식 계획을 조율하고 제안하세요.
    """
)


## Test


In [11]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="저는 런던에 살고 있고, 파리에서 하객 100명 규모로 jazz 장르 음악으로 결혼식을 하고 싶어요.")],
    }
)

In [12]:
from pprint import pprint

pprint(response)

{'destination': 'Paris',
 'genre': 'Jazz',
 'guest_count': '100',
 'messages': [HumanMessage(content='저는 런던에 살고 있고, 파리에서 하객 100명 규모로 jazz 장르 음악으로 결혼식을 하고 싶어요.', additional_kwargs={}, response_metadata={}, id='ff31ebac-20cf-420e-9758-5329a77cb36c'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 868, 'prompt_tokens': 371, 'total_tokens': 1239, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D6udpFZopj4FOw8ZyBDF74hUMMd3P', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3c69-6c99-7ba1-86b2-2ac838f97960-0', tool_calls=[{'name': 'update_state', 'args': {'origin': 'London', 'destination': 'Paris', 'g

In [13]:
print(response["messages"][-1].content)

상태 업데이트 완료: 출발지 London, 도착지 Paris, 게스트 수 100명, 음악 장르 Jazz로 계획 중입니다.

다음은 각 전문가에게 위임한 업무의 현재 상황 요약과 제안입니다. 추가 질문 없이 바로 실행 가능한 형태로 정리합니다.

1) 항공권(Travel/항공권 전문가) 위임 결과 요약
- 현재 제시된 대략적 그룹 항공 옵션이 있습니다. 주요 포인트
  - LHR → ORY(파리 오를리) 직항 다수
    - 예시 일정 1: 2026-01-06 15:00 → 17:35, 약 1h35m, 약 51 EUR
    - 예시 일정 2: 2026-01-10 15:00 → 17:35, 약 1h35m, 약 48 EUR
    - 예시 일정 3: 2026-09-15 15:00 → 17:35, 약 1h35m, 약 51 EUR
  - LTN/LGW/STN 등 런던 인근 공항에서 CDG/ORY로의 대안 옵션도 포함
    - STN → CDG: 2026-05-15 17:00 → 19:10, 약 1h10m, 약 59 EUR
    - LGW → CDG: 2026-10-01 18:00 → 19:20, 약 1h20m, 약 68 EUR
  - 요약: 파리의 ORY/CDG로의 직항 위주, 1시간 20–1시간 40분 내외의 비행 시간 및 48–68 EUR대의 편도 가격 후보 다수. 그룹 예약 시 5–6주 전후 여유를 두고 확정하는 방식이 일반적이며, 좌석 배치(100석 규모) 및 수하물 포함 여부에 따라 가격 차이가 생깁니다.
- 위 제안은 현재 수집된 후보들로 구성되며, 최종 확정은 날짜(Wedding date), 도착/출발 분산 여부, 수하물 포함 여부, 좌석 블록 수 등에 따라 결정됩니다.
- 담당 항공권 전문가에게 위임할 구체적 작업(요청 포맷 예시)
  - 100명 규모의 그룹 항공 예약 패키지 제안: 최소 2개 날짜(예: 결혼식 전날 도착/당일 도착)로 나눠 견적 확보
  - 좌석 배치: 같은 항공편에 가능한 한 다수의 게스트가 함께 도착하도록 묶기
  - 공항-베

link to trace: https://smith.langchain.com/public/7b5fe668-d3e3-4af4-b513-a8cacc0c9e84/r